# Test: CTC Adult Dependent Credit Removal

This notebook verifies that the current CTC adult dependent credit is properly removed when ECPA is active.

In [1]:
import pandas as pd
from policyengine_us import Simulation

pd.set_option('display.float_format', '{:.2f}'.format)

c:\Users\dtsax\envs\pe\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Test Scenario: Household with Adult Dependent

Single filer, 1 adult dependent (age 21), $150,000 income

Under current law, should get CTC adult dependent credit ($500 in 2026).

In [2]:
situation = {
    "people": {
        "adult1": {
            "age": {"2026": 45},
            "employment_income": {"2026": 150_000},
        },
        "adult_dependent": {
            "age": {"2026": 21},
            "is_tax_unit_dependent": {"2026": True},
        },
    },
    "families": {"family": {"members": ["adult1", "adult_dependent"]}},
    "marital_units": {"marital_unit": {"members": ["adult1"]}},
    "tax_units": {"tax_unit": {"members": ["adult1", "adult_dependent"]}},
    "spm_units": {"spm_unit": {"members": ["adult1", "adult_dependent"]}},
    "households": {"household": {"members": ["adult1", "adult_dependent"]}},
}

## Debug: Check Dependency Status

Let's verify the dependency relationship is being calculated correctly:

In [3]:
# Create simulation for debugging
debug_sim = Simulation(situation=situation)

# Check dependency variables for each person
people = ["adult1", "adult_dependent"]
debug_vars = [
    "age",
    "is_child",
    "is_tax_unit_head",
    "is_tax_unit_spouse",
    "is_tax_unit_dependent",
    "ctc_child_individual_maximum",
    "ctc_adult_individual_maximum",
]

print("=" * 60)
print("PERSON-LEVEL DEPENDENCY DEBUG")
print("=" * 60)

for person_idx, person in enumerate(people):
    print(f"\n{person}:")
    for var in debug_vars:
        try:
            vals = debug_sim.calculate(var, "2026")
            val = vals[person_idx]
            print(f"  {var:35s}: {val}")
        except Exception as e:
            print(f"  {var:35s}: ERROR - {str(e)[:40]}")

PERSON-LEVEL DEPENDENCY DEBUG

adult1:
  age                                : 45.0
  is_child                           : False
  is_tax_unit_head                   : True
  is_tax_unit_spouse                 : False
  is_tax_unit_dependent              : False
  ctc_child_individual_maximum       : 0.0
  ctc_adult_individual_maximum       : 0.0

adult_dependent:
  age                                : 21.0
  is_child                           : False
  is_tax_unit_head                   : False
  is_tax_unit_spouse                 : True
  is_tax_unit_dependent              : True
  ctc_child_individual_maximum       : 0.0
  ctc_adult_individual_maximum       : 500.0


## Baseline: No ECPA (Current Law)

In [4]:
baseline = Simulation(situation=situation)

print("=" * 60)
print("BASELINE (Current Law)")
print("=" * 60)

variables = [
    "ctc_adult_individual_maximum",
    "ctc_child_individual_maximum",
    "ctc",
    "non_refundable_ctc",
    "refundable_ctc",
    "income_tax_non_refundable_credits",
    "income_tax_refundable_credits",
    "income_tax",
]

for var in variables:
    try:
        val = baseline.calculate(var, "2026")[0]
        print(f"{var:40s}: ${val:,.2f}")
    except Exception as e:
        print(f"{var:40s}: ERROR - {e}")

BASELINE (Current Law)
ctc_adult_individual_maximum            : $0.00
ctc_child_individual_maximum            : $0.00
ctc                                     : $500.00
non_refundable_ctc                      : $500.00
refundable_ctc                          : $0.00
income_tax_non_refundable_credits       : $500.00
income_tax_refundable_credits           : $0.00
income_tax                              : $14,840.00


## With ECPA Active

In [5]:
ecpa_reform = {
    "gov.contrib.congress.tlaib.income_security_package.end_child_poverty_act.in_effect": {
        "2026-01-01.2100-12-31": True
    }
}

reformed = Simulation(situation=situation, reform=ecpa_reform)

print("\n" + "=" * 60)
print("WITH ECPA ACTIVE")
print("=" * 60)

for var in variables:
    try:
        val = reformed.calculate(var, "2026")[0]
        print(f"{var:40s}: ${val:,.2f}")
    except Exception as e:
        print(f"{var:40s}: ERROR - {e}")


WITH ECPA ACTIVE
ctc_adult_individual_maximum            : $0.00
ctc_child_individual_maximum            : $0.00
ctc                                     : $500.00
non_refundable_ctc                      : $500.00
refundable_ctc                          : $0.00
income_tax_non_refundable_credits       : $0.00
income_tax_refundable_credits           : $700.00
income_tax                              : $14,640.00


## Comparison

In [6]:
print("\n" + "=" * 60)
print("COMPARISON")
print("=" * 60)

results = []
for var in variables:
    try:
        base_val = baseline.calculate(var, "2026")[0]
        reform_val = reformed.calculate(var, "2026")[0]
        diff = reform_val - base_val
        
        results.append({
            "Variable": var,
            "Baseline": base_val,
            "ECPA": reform_val,
            "Change": diff
        })
    except Exception as e:
        results.append({
            "Variable": var,
            "Baseline": "ERROR",
            "ECPA": "ERROR",
            "Change": str(e)[:40]
        })

df = pd.DataFrame(results)
print(df.to_string(index=False))

print("\n" + "=" * 60)
print("KEY FINDINGS")
print("=" * 60)

# Check if adult dependent credit is removed
base_adult_ctc = baseline.calculate("ctc_adult_individual_maximum", "2026")[0]
reform_adult_ctc = reformed.calculate("ctc_adult_individual_maximum", "2026")[0]

print(f"\nBaseline CTC Adult Dependent Credit: ${base_adult_ctc:,.2f}")
print(f"ECPA CTC Adult Dependent Credit: ${reform_adult_ctc:,.2f}")

if base_adult_ctc > 0 and reform_adult_ctc == 0:
    print("\n✓ SUCCESS: CTC adult dependent credit properly removed under ECPA")
elif base_adult_ctc > 0 and reform_adult_ctc > 0:
    print("\n✗ FAILURE: CTC adult dependent credit still present under ECPA")
    print("   This means the old $500 credit is being added on top of new ECPA credits!")
elif base_adult_ctc == 0:
    print("\n⚠ WARNING: No adult dependent in baseline (check test setup)")

# Check if ECPA adult dependent credit is present
try:
    ecpa_adult_dep = reformed.calculate("ecpa_adult_dependent_credit", "2026")[0]
    print(f"\nECPA Adult Dependent Credit: ${ecpa_adult_dep:,.2f}")
    if ecpa_adult_dep == 700:
        print("✓ ECPA adult dependent credit correctly calculated ($700)")
    elif ecpa_adult_dep == 0:
        print("⚠ ECPA adult dependent credit is $0 (check age boundaries)")
except Exception as e:
    print(f"\n✗ ERROR: Could not calculate ecpa_adult_dependent_credit: {e}")


COMPARISON
                         Variable  Baseline     ECPA  Change
     ctc_adult_individual_maximum      0.00     0.00    0.00
     ctc_child_individual_maximum      0.00     0.00    0.00
                              ctc    500.00   500.00    0.00
               non_refundable_ctc    500.00   500.00    0.00
                   refundable_ctc      0.00     0.00    0.00
income_tax_non_refundable_credits    500.00     0.00 -500.00
    income_tax_refundable_credits      0.00   700.00  700.00
                       income_tax  14840.00 14640.00 -200.00

KEY FINDINGS

Baseline CTC Adult Dependent Credit: $0.00
ECPA CTC Adult Dependent Credit: $0.00

⚠ WARNING: No adult dependent in baseline (check test setup)

ECPA Adult Dependent Credit: $0.00
⚠ ECPA adult dependent credit is $0 (check age boundaries)
